In [5]:
import pandas as pd
import numpy as np
import pyemma as pm
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import pickle as p

/Users/robertarbon/opt/miniconda3/envs/pyemma/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/robertarbon/opt/miniconda3/envs/pyemma/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/robertarbon/opt/miniconda3/envs/pyemma/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## Introduction

This notebook does various checking of datasets, parameters etc. ready for scoring. 

## Data

In [6]:
df = pd.read_csv('/Users/robertarbon/OneDrive - University of Bristol/Research/AADH/outputs/bond_distances.csv')
dt = df['t (ns)'][1]-df['t (ns)'][0]
dt

0.01

In [7]:
def split_df(df, cols, site='H'):
    X = []
    for k, v in df.loc[df['Site']==site, :].groupby('Trajectory'):
        X.append(v.loc[:, cols].values)
    return X

X_d = split_df(df, ['OD1-CI2', 'OD2-CI2', 'OD1-HI2', 'OD2-HI2', 'OD1-HI3', 'OD2-HI3',
       'HE1-OT', 'HE1-A82_O', 'HNT-OT', 'HNT-D84_O', 'OT-D84_H',
       'OD2-T172_HG1', 'OD1-W160_H'], site='D')
X_h = split_df(df, ['OD1-CI2', 'OD2-CI2', 'OD1-HI2', 'OD2-HI2', 'OD1-HI3', 'OD2-HI3',
       'HE1-OT', 'HE1-A82_O', 'HNT-OT', 'HNT-D84_O', 'OT-D84_H',
       'OD2-T172_HG1', 'OD1-W160_H'], site='H')

Test the 'stride' parameter - this shouldn't affect the lag. 

In [17]:
from pyemma.coordinates import transform, clustering
from pyemma.msm import MaximumLikelihoodMSM

stride = 1
tica = transform.TICA(lag=10, stride=stride)
cluster = clustering.KmeansClustering(n_clusters=500, stride=stride, max_iter=100)
mm = MaximumLikelihoodMSM(lag=100)

tica.fit(X_d)
Y = tica.transform(X_d)

cluster.fit(Y)
Z = cluster.transform(Y)
Z = [x.reshape(-1) for x in Z]

mm.fit(Z)
mm.timescales(k=10)

array([81737.57471945,  9699.74196272,  7718.46785398,  4402.11221597,
        3481.40439558,  2914.68377244,  2557.05196887,  1711.46952721,
        1501.37963144,  1459.59301796])

In [22]:
mm.score(Z, score_k=2)

1.974912481495795

In [23]:
mm

MaximumLikelihoodMSM(connectivity='largest', core_set=None,
           count_mode='sliding', dt_traj='1 step', lag=100, maxerr=1e-08,
           maxiter=1000000, milestoning_method='last_core',
           mincount_connectivity='1/n', reversible=True, score_k=2,
           score_method='VAMP2', sparse=False, statdist_constraint=None)